Code for selecting sets of tags and obtaining the PMIDs. Then, these can be tagged in Paperpile for easy access in Boox.

In [1]:
import logging
import numpy as np

import os
os.chdir("..")

import pandas as pd

from src.db import Database
from src.tags import Tags

LOGGER = logging.getLogger(__name__)
logging.basicConfig(level = logging.INFO)

In [40]:
def load_metadata(db, filter=True):
    # Initialize Tag class
    metadata = Tags()

    # Load the tag file and turn into a dataframe
    metadata.load("data/tag_files/tags.yaml")
    md = pd.DataFrame(metadata.tags["tagged_papers"])
    
    # Merge with database
    mddb = db.merge(md, on="id", how="outer")
    
    # List current progress out of total.
    total_papers = mddb.shape[0]
    tagged_papers = sum(~mddb["tag"].isnull())
    percent = np.round((tagged_papers/total_papers) * 100, 2)
    
    LOGGER.info(f"Currently {tagged_papers}/{total_papers} ({percent}%) papers tagged.")
    LOGGER.info(f"{total_papers-tagged_papers} papers remaining.")
        
    mddb = mddb.reset_index()
        
    return mddb

def filter_by_tags(mddb, tag_list: list):
    """ Returns papers containing the tagged term """
    current_tags = mddb["tag"].to_list()
    tags_to_filter = []
    for idx, tags in enumerate(current_tags):
        matches = set(tag_list).intersection(tags)
        if len(matches) == 0:
            pass
        else:
            tags_to_filter.append(idx)
    
    filtered = mddb.iloc[tags_to_filter]
    LOGGER.info(f"{len(filtered)} matches found.")
    return filtered[["id", "tag", "title"]]

In [17]:
# Set tables
tables = ["active_inference", "bayesian_mechanics", "free_energy", "friston", "karl_friston", "predictive_coding", "predictive_processing"]

# Load database
database = Database()
database.load(tables=tables)
db = database.db

# Load metadata
mddb = load_metadata(db, filter=False)

INFO:src.db:Checking tables...
INFO:src.db:Loading tables...
INFO:src.db:Tables downloaded from PubMed on Thursday, Sept. 14, 2023.
INFO:src.tags:YAML tag file successfully loaded from data/tag_files/tags.yaml.
INFO:__main__:Currently 3585/3585 (100.0%) papers tagged.
INFO:__main__:0 papers remaining.


## High-level tag sets

[core ]

**Robotics**
[robotics, ]
[brain-computer interface, human-machine interface]
[deep active inference ]

**Neuropathology and disease**

[addiction ]
[Alzheimer's disease, Huntington's disease, down syndrome, multiple sclerosis, Parkinson's disease]
[anxiety, panic disorder ]
[apathy, catatonia, depression, mood]
[anosognosia ]
[autism spectrum disorder, ]
[bipolar disorder, borderline personality disorder, delusion, depersonalization, hallucination, obsessive-compulsive disorder]
[illusion ]
[cancer ]
[clinical psychology ]
[computational psychiatry ]
[disconnection ]
[dyslexia ]
[eating disorder, obesity ]
[epilepsy ]
[headache ]
[hypnosis ]
[hysteria ]

**Behavior**
[animal behavior, anticipation, attachment theory, human behavior]
[attention, precision]
[decision making, goal-directed behavior, motivation]
[exploration, artificial curiosity, information seeking, insight ]
[habit learning, learning]
[navigation]

**Perception and general brain function**

[aesthetic perception, anaesthesia, anosognosia, arousal, binocular rivalry]
[audition ]
[vision ]
[cognitive control, economics ]
[belief formation, ]
[default mode network ]
[degeneracy ]
[emotion, empathy ]
[existential threat ]
[intelligence ]
[lateralization ]
[lesions ]
[linguistics ]
[memory ]
[metacognition ]
[mirror-neuron ]
[multimodal integration]
[neural coding]
[neural substrate]
[neuroanatomy]
[neurocircuitry]
[neurocomputation]
[neuroimaging]
[neuromodulation]
[neuronal dynamics]
[neuronal populations]
[nociceptive processing ]
[noise]
[normativity]
[pain]
[perception]
[placebo]

**Vision**
[object localization]

**Movement and coordination**
[dysmetria ]
[forward model ]
[movement]

**Consciousness, identity, and selfhood**

[anaesthesia, anosognosia]
[consciousness ]
[integrated information theory ]
[interoceptive processing ]
[meditation ]

**General biology**

[aging, child development, embryogenesis, morphogenesis]
[immune system, endocrinology, chemical reaction networks, metabolism]
[injury ]

**Agent-environment interaction**

[adaptation, affordances, agent-environment coupling, brain-body interaction, control theory, cybernetics, ecological psychology, embodied cognition]
[niche construction]
[homeostasis / allostasis, autopoiesis, homeostatic reinforcement learning]
[enactive inference, enactivism ]
[energy homeostasis principle ]
[exteroceptive processing ]

**Collective behavior and dynamics**
[ecosystems ] 
[criticality ]
[collective behavior, communication, organizational ]
[culture ]
[evolution ]
[mirror-neuron ]

**New frontiers**
[covid-19 ]
[ethics ]
[military research ]

**Critique and commentary**
[philosophy]
[editorial]
[comment / response]

**Theory**
[Bayesian filtering, ]
[Bayesian model reduction, Bayesian model selection]
[Bethe free energy, Kikuchi's free energy]
[branching-time active inference ]
[dynamic causal modeling ]
[expected free energy ]
[explainability ]
[free energy of the expected future ]
[generalized free energy ]
[generalized predictive coding ]
[graphical models ]
[hierarchical Gaussian filter ]
[hierarchical model ]
[homeostatic reinforcement learning ]
[information geometry, information theory, expected float entropy ]
[KL control]
[lattice theory]
[logic]
[message passing]
[morphological computation]
[network analysis]
[neuromorphic computing]
[neuronal gauge theory]
[policy pruning]

**Bayesian mechanics**
[Markov blankets, Bayesian mechanics ]
[path-integral formulation]

In [41]:
tag_list = ["robotics"]
filtered = filter_by_tags(mddb, tag_list=tag_list)

INFO:__main__:34 matches found.


In [42]:
filtered

,id,tag,title
25,35327872,"[robotics, review]",How Active Inference Could Help Revolutionise ...
52,34022669,"[deep active inference, robotics]",Robot navigation as hierarchical active inference
59,34901166,"[robotics, social, emotion, multimodal integra...",Active Inference Through Energy Minimization i...
88,33304260,"[robotics, deep active inference, discrete sta...",Learning Generative State Space Models for Act...
119,27683002,"[robotics, psychopathology]",Active inference and robot control: a case study
128,34720895,"[robotics, human-machine interface, hierarchic...",Trust as Extended Control: Human-Machine Inter...
163,33335499,"[robotics, human-machine interface, social]",A Hybrid Human-Neurorobotics Approach to Prima...
173,33500908,"[evolution, robotics, ecological psychology]",The Active Inference Approach to Ecological Pe...
174,34343777,[robotics],Bidirectional interaction between visual and m...
213,32390817,"[emotion, review, robotics, human-machine inte...",An Investigation of the Free Energy Principle ...


In [37]:
filtered["id"].to_list()

[35327872,
 34022669,
 34901166,
 33304260,
 27683002,
 34720895,
 33335499,
 33500908,
 34343777,
 32390817,
 33266972,
 33013346,
 34496394,
 33746730,
 34412116,
 35455132,
 35496899,
 33286336,
 35966370,
 30686712,
 34742776,
 36832633,
 36246500,
 29903532,
 37256968,
 34634605,
 36605522,
 34682030,
 33041778,
 33652371,
 21515639,
 33891957,
 32903328,
 34601363]

In [39]:
mddb

,index,id,title,authors,citation,first_author,journal_book,publication_year,create_date,pmcid,nihms_id,doi,tag
0,0,34968557,Active inference leads to Bayesian neurophysio...,Isomura T.,Neurosci Res. 2022 Feb;175:38-45. doi: 10.1016...,Isomura T,Neurosci Res,2022,2021/12/30,NaN,NaN,10.1016/j.neures.2021.12.003,[neural substrate]
1,1,33343039,Active inference on discrete state-spaces: A s...,"Da Costa L, Parr T, Sajid N, Veselic S, Neacsu...",J Math Psychol. 2020 Dec;99:102447. doi: 10.10...,Da Costa L,J Math Psychol,2020,2020/12/21,PMC7732703,NaN,10.1016/j.jmp.2020.102447,"[theory, discrete state space / Markov decisio..."
2,2,30120751,"Active Inference, Novelty and Neglect","Parr T, Friston KJ.",Curr Top Behav Neurosci. 2019;41:115-128. doi:...,Parr T,Curr Top Behav Neurosci,2019,2018/08/19,NaN,NaN,10.1007/7854_2018_61,"[vision, saccades, neuroanatomy, discrete stat..."
3,3,27375276,Active inference and learning,"Friston K, FitzGerald T, Rigoli F, Schwartenbe...",Neurosci Biobehav Rev. 2016 Sep;68:862-879. do...,Friston K,Neurosci Biobehav Rev,2016,2016/07/05,PMC5167251,NaN,10.1016/j.neubiorev.2016.06.022,"[theory, learning, discrete state space / Mark..."
4,4,34687699,"Active inference, selective attention, and the...","Holmes E, Parr T, Griffiths TD, Friston KJ.",Neurosci Biobehav Rev. 2021 Dec;131:1288-1304....,Holmes E,Neurosci Biobehav Rev,2021,2021/10/23,PMC8643962,NIHMS1754010,10.1016/j.neubiorev.2021.09.038,"[attention, application, event related potential]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3580,3580,36601626,Changes in both top-down and bottom-up effecti...,"Thomas GEC, Zeidman P, Sultana T, Zarkali A, R...",Brain Commun. 2022 Dec 14;5(1):fcac329. doi: 1...,Thomas GEC,Brain Commun,2022,2023/01/05,PMC9798302,NaN,10.1093/braincomms/fcac329,[predictive processing]
3581,3581,28330464,Spectral-temporal EEG dynamics of speech discr...,"Gilley PM, Uhler K, Watson K, Yoshinaga-Itano C.",BMC Neurosci. 2017 Mar 22;18(1):34. doi: 10.11...,Gilley PM,BMC Neurosci,2017,2017/03/24,PMC5439120,NaN,10.1186/s12868-017-0353-4,[predictive processing]
3582,3582,36382066,A robot or a dumper truck? Facilitating play-b...,"Paldam E, Roepstorff A, Steensgaard R, Lundsga...",Autism Dev Lang Impair. 2022 Mar 21;7:23969415...,Paldam E,Autism Dev Lang Impair,2022,2022/11/16,PMC9620708,NaN,10.1177/23969415221086714,[predictive processing]
3583,3583,31678780,The neurophenomenology of early psychosis: An ...,"Nelson B, Lavoie S, Gawęda Ł, Li E, Sass LA, K...",Conscious Cogn. 2020 Jan;77:102845. doi: 10.10...,Nelson B,Conscious Cogn,2020,2019/11/04,NaN,NaN,10.1016/j.concog.2019.102845,[predictive processing]
